In [35]:
from ckks_package.ckks import *

In [36]:
N = 2**15  # Ring degree
n = 2**3  # Number of slots
L_boot = 19  # Maximal level during bootstrapping
q0 = 2**29  # Smallest modulus
p = 2**23  # Scaling factor outside of bootstrapping
delta = q0  # Scaling factor during bootstrapping

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L_boot, q0, p, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(91.9) operations.

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [37]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([ 2.+1.j,  3.-1.j,  3.-2.j,  1.+3.j, -2.-1.j, -1.-1.j,  0.+2.j,
         2.-3.j]),
 array([ 1.+1.j,  3.-2.j,  2.+3.j,  2.-3.j, -3.-3.j,  2.+0.j,  3.+3.j,
        -0.-2.j])]

In [38]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[8388608 + 7213284X^2048 - 4442517X^4096 - 10662270X^6144 + 9639050X^10240 - 1370031X^12288 + 8272721X^14336 - 2097152X^16384 + 4666764X^18432 + 2975121X^20480 - 3945957X^22528 + 2965821X^24576 - 6323507X^26624 + 567485X^28672 - 5158284X^30720 mod(2^581),
 10485760 + 5892528X^2048 - 8289031X^4096 + 5581956X^6144 + 5190186X^8192 + 5354942X^10240 - 8856516X^12288 + 3630177X^14336 - 3145728X^16384 + 523297X^18432 - 9051246X^20480 + 6142777X^22528 + 11121828X^24576 + 317092X^26624 + 7681216X^28672 - 6578313X^30720 mod(2^581)]

In [39]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).,
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).]

In [40]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 1.99999970e+00+1.00000035j  2.99999977e+00-0.99999898j
  3.00000026e+00-2.00000087j  9.99999324e-01+3.00000029j
 -2.00000021e+00-0.99999746j -9.99998128e-01-1.00000017j
 -2.96213050e-07+2.00000082j  1.99999862e+00-3.00000113j]
[ 9.99999836e-01+9.99999987e-01j  3.00000100e+00-1.99999954e+00j
  1.99999980e+00+2.99999989e+00j  2.00000155e+00-3.00000048e+00j
 -2.99999790e+00-2.99999952e+00j  2.00000191e+00+1.47353414e-06j
  2.99999965e+00+3.00000093e+00j -1.16227049e-07-1.99999989e+00j]


In [41]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^23 (level 23 out of 24).


In [42]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 2.99999954+2.00000034e+00j  6.00000077-2.99999852e+00j
  5.00000006+9.99999025e-01j  3.00000088-1.85107446e-07j
 -4.99999811-3.99999698e+00j  1.00000378-9.99998693e-01j
  2.99999935+5.00000175e+00j  1.9999985 -5.00000102e+00j]
[ 1.00000005+2.9999997j   7.00000433-8.99999561j 12.00000157+4.99999959j
 11.00000194+3.00000784j  3.00000511+8.99999132j -1.99999696-2.00000329j
 -6.00000548+6.00000084j -6.00000198-3.99999658j]


In [43]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt_boot = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt_boot))

print(ct.get_poly_precision(ct_boot, CKKS.sk))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^0 (level 0 out of 24).
1
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^1 (level 1 out of 24).
[ 1.99751132e+00+0.99804821j  2.99356646e+00-0.99808391j
  2.99544531e+00-1.99540353j  1.00031083e+00+2.99507654j
 -2.00022703e+00-0.99849616j -9.96798977e-01-1.00109535j
  1.02095093e-03+1.99498389j  1.99709856e+00-2.99558758j]
9.36885307018454
